Scoping the feasibility for migrant phenotype project

<u>Process</u>
    
    - list of project codes as csv 
    - map from read to snomend ct 
    - map from snomed ct to gdppr

Gotcha's

 - Multiple read_codes point to the same concept

 - Singular read_codes point to different concept

 - many-many relationship

 - Mapping file has columns IS_ASSURED EffectiveDate MapStatus

 - Filter applied on mapping file was:
 
 
     - IS_ASSURED == 1
     - EffectiveDate == most recent date
     - MapStatus == 1

In [1]:
import pandas as pd
import os
os.chdir('../')
pd.set_option('display.max_columns', None)

In [2]:
os.getcwd()

'/Users/zachwelshman/Developer/Github/BHF_DSC_HDS/ZW/migrant_feasibility'

Takes a PDF that has been coverted to csv using adobe online converter and formatted, as a minimum below
___________


| read_code |
| --- | 
|9Nm6.00 |

The output should contain something like:
___________


| read_code | read_code_mapfile | map_id | concept_id_mapfile | concept_id_gdppr | CONCEPT_MAP_FLAG | GPDDR_MAP_FLAG |
| --- | --- | --- | --- | --- | --- | --- | 
|9Nm6.00 | 9Nm6.00 | {f9b73a54-2623-11e3-a0b5-00ff3a5bce8f} | 359821000000104 | 359821000000104 | 1 | 1




Creating python file from Jupyter to make version control and portabilty easier

In [3]:
from src.readv2_to_snomed_search_gdppr import *
import pandas as pd
import os


def main(project_csv,
         project_codelist_column,
         mapping_file,
         mapping_file_codelist_column,
         mapped_snomed_project_codelist_column,
         mapping_file_gdppr_codelist_column,
         output_prefix
         ):
    # map project codes to readv2 map for table 1

    read_to_snomed_obj = readv2ToSnomedConversion(
        project_df=project_csv,
        project_codelist_column=project_codelist_column,
        mapping_file=mapping_file,
        mapping_file_codelist_column=mapping_file_codelist_column)

    project_read_to_snomed_df = read_to_snomed_obj.map_readV2_to_snomed()
    print(project_read_to_snomed_df.nunique())

    #   refactor output location and name into main parameter.
    project_read_to_snomed_df.to_csv(
        f'output_data/{output_prefix}_read_v2_to_snomedct.csv')

    print('For read_v2_to_snomedct mapping')
    conversionChecks.get_codes_not_mapped(project_csv,
                                          project_codelist_column,
                                          project_read_to_snomed_df,
                                          mapping_file_codelist_column,
                                          map_flag_col='READV2_MAP_FLAG')

    #   refactor to bring the mapping file ingestion into a parameter.
    gdppr_codes_clusters = pd.read_csv(
        'mapping_files/GDPPR_Cluster_Refset_1000230_20220429.csv',
        encoding='unicode_escape')

    # map project_readv2 to gdppr
    project_read_to_snomed_df['ConceptId'] = project_read_to_snomed_df['ConceptId'].astype(
        'Int64')
    gdppr_codes_clusters['ConceptId'] = gdppr_codes_clusters['ConceptId'].astype(
        'Int64')

    snomed_to_gdppr_obj = snomedToGdpprLookUp(
        project_read_to_snomed_df,
        mapped_snomed_project_codelist_column=mapped_snomed_project_codelist_column,
        mapping_file=gdppr_codes_clusters,
        mapping_file_gdppr_codelist_column=mapping_file_gdppr_codelist_column)

    snomed_to_gdppr_df = snomed_to_gdppr_obj.map_snomed_to_gdppr()
    snomed_to_gdppr_df = snomed_to_gdppr_obj.sort_effective_date_and_assured(
        snomed_to_gdppr_df)
    print(snomed_to_gdppr_df.nunique())

    print('''For snomedct_to_gdppr mapping
                adding a change''')
    conversionChecks.get_codes_not_mapped(
        project_read_to_snomed_df,
        mapped_snomed_project_codelist_column,
        snomed_to_gdppr_df,
        mapping_file_gdppr_codelist_column,
        map_flag_col='GDPPR_MAP_FLAG')

    #   refactor output location and name into main parameter.
    snomed_to_gdppr_df.to_csv(
        f'output_data/{output_prefix}_snomedct_to_gdppr.csv')

    # formatting output for reseacher convience.
    # format_output(researcher_dir,
    #               researcher_file,
    #               output_dir = f'output_data/',
    #               output_file =  f'{output_prefix}_snomedct_to_gdppr.csv'),
    #               left_on,
    #               right_on)


def get_project_code_list_1(data_dir='input_data'):
    '''
    Reads the codelist as an excel (.xlsx) file.
    Args:
        data_dir:

    Returns:
        the reseachers codelist

    '''
    converted_project_codelist_1 = pd.read_excel(
        # pass the sheet_name to input param
        f'{data_dir}/codelist_converted.xlsx', sheet_name='table_1')
    converted_project_codelist_1['read_code'] = converted_project_codelist_1['read_code'].astype(
        str)
    return converted_project_codelist_1

# replacing some code is specific to the migrant phenotyping work


def get_project_code_list_2(data_dir='input_data'):
    converted_project_codelist_2 = pd.read_excel(
        # pass the sheet_name to input param
        f'{data_dir}/codelist_converted.xlsx', sheet_name='table_2')
    converted_project_codelist_2['read_code'] = converted_project_codelist_2['read_code'].astype(
        str)
    converted_project_codelist_2['read_code'].replace(
        '6951', '6951.00', inplace=True)
    converted_project_codelist_2['read_code'].replace(
        '1695', '1695.00', inplace=True)
    converted_project_codelist_2['read_code'].replace(
        '1343', '1343.00', inplace=True)
    converted_project_codelist_2['read_code'].replace(
        '1345', '1345.00', inplace=True)
    converted_project_codelist_2['read_code'].replace(
        '1344', '1344.00', inplace=True)
    converted_project_codelist_2['read_code'].replace(
        '1342', '1342.00', inplace=True)
    converted_project_codelist_2['read_code'].replace(
        '1347', '1347.00', inplace=True)
    converted_project_codelist_2['read_code'].replace(
        '1348', '1348.00', inplace=True)
    converted_project_codelist_2['read_code'].replace(
        '1346', '1346.00', inplace=True)
    converted_project_codelist_2['read_code'].replace(
        '2263', '2263.00', inplace=True)
    return converted_project_codelist_2


def prepare_mapping_file(data_dir='mapping_files'):
    read_snomed_map = pd.read_csv(
        f'{data_dir}/rcsctmap2_uk_20200401000001.txt',
        sep='\t',
        encoding='unicode_escape')
    prepared_read_snomed_map = (projectDataPreparation
                                .create_read_term_column(read_snomed_map,
                                                         'ReadCode',
                                                         'TermCode',
                                                         'read_term_map'))
    return prepared_read_snomed_map

# def format_output(researcher_dir ,researcher_file, output_dir, output_file, left_on, right_on):
#     '''
#
#     Args:
#         researcher_dir: codelist_dir
#         researcher_file: codelist_file
#         output_dir: output directory for the final mapping result
#         output_file: final mapping result
#         left_on: left join column
#         right_on: right join column
#
#
#     Returns:
#
#     '''
#     researcher_file = pd.read_excel(os.path.join(researcher_dir, researcher_file))
#     output_file pd.read_excel(os.path.join(output_dir,output_file))
#     return researcher_file.merge(output_file, left_on, right_on, how='left')


# Pass through arg_parse: see ccu_dq repo for an example
main(project_csv=get_project_code_list_1(),
     project_codelist_column='read_code',
     mapping_file=prepare_mapping_file(),
     mapping_file_codelist_column='read_term_map',
     mapped_snomed_project_codelist_column='ConceptId',
     mapping_file_gdppr_codelist_column='ConceptId',
     output_prefix='table_1')



6_level_grouping       6
18_level_grouping     19
read_code            168
read_term            167
medcode              168
MapId                183
ReadCode             149
TermCode               4
ConceptId            162
DescriptionId        168
IS_ASSURED             2
EffectiveDate          7
MapStatus              2
read_term_map        168
READV2_MAP_FLAG        1
dtype: int64
For read_v2_to_snomedct mapping


        Successfully mapped 168 codes from mapped_file of
         168 codes found in the input_df

        
6_level_grouping           6
18_level_grouping         19
read_code                166
read_term                165
medcode                  166
MapId                    179
ReadCode                 147
TermCode                   4
ConceptId                162
DescriptionId            168
IS_ASSURED                 1
EffectiveDate              7
MapStatus                  1
read_term_map            166
READV2_MAP_FLAG            1
Cluster_Category           1
Clust

In [4]:
main(project_csv = get_project_code_list_2(),
     project_codelist_column='read_code',
     mapping_file=prepare_mapping_file(),
     mapping_file_codelist_column='read_term_map',
     mapped_snomed_project_codelist_column='ConceptId',
     mapping_file_gdppr_codelist_column='ConceptId',
     output_prefix='table_2')

level_of_certainty_of_migration_status      3
type_of_code                                4
read_code                                 434
read_term                                 431
medcode                                   434
MapId                                     545
ReadCode                                  425
TermCode                                    5
ConceptId                                 506
DescriptionId                             526
IS_ASSURED                                  2
EffectiveDate                              12
MapStatus                                   2
read_term_map                             434
READV2_MAP_FLAG                             1
dtype: int64
For read_v2_to_snomedct mapping


        Successfully mapped 434 codes from mapped_file of
         434 codes found in the input_df

        
level_of_certainty_of_migration_status      3
type_of_code                                4
read_code                                 433
read_term       

In [5]:
# format outputs
table_1 = pd.read_csv('output_data/table_1_snomedct_to_gdppr.csv')
table_1_cols_to_keep = ['6_level_grouping', 
                         '18_level_grouping',
                         'read_code',
                         'read_term',
                         'medcode',
                         'READV2_MAP_FLAG',
                         'ConceptId',
                         'Cluster_Category', 
                         'Cluster_ID',
                         'Cluster_Desc',
                         'ConceptId_Description',
                         'GDPPR_MAP_FLAG']

table_1 = table_1[table_1_cols_to_keep ]

In [6]:
reseacher_codelist = get_project_code_list_1()
formatted_df = reseacher_codelist.merge(table_1, left_on ='read_code', right_on ='read_code', how='left', 
                         suffixes=('_original',''))

cols_to_drop = [i for i in formatted_df.columns if 'original' in i]
formatted_df.drop(columns = cols_to_drop, inplace=True)
formatted_df = formatted_df[table_1_cols_to_keep]

In [7]:
formatted_df.to_csv('output_data/formatted_table_1_snomedct_to_gdppr.csv', index=False)

In [8]:
formatted_df.nunique()

6_level_grouping           6
18_level_grouping         19
read_code                168
read_term                165
medcode                  166
READV2_MAP_FLAG            1
ConceptId                162
Cluster_Category           1
Cluster_ID                13
Cluster_Desc              13
ConceptId_Description    157
GDPPR_MAP_FLAG             2
dtype: int64

In [9]:
formatted_df.sum()

/var/folders/h5/x2gkzm3j06z5k0dqwzgk7wvw0000gn/T/ipykernel_2506/1585014424.py:1: FutureWarning: The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  formatted_df.sum()


read_code          9S10.009S13.009S14.009i00.009i20.009i21.009i22...
medcode                                                    5893204.0
READV2_MAP_FLAG                                                179.0
ConceptId                                         9367852095587822.0
GDPPR_MAP_FLAG                                                 171.0
dtype: object

In [10]:
print(f"""{formatted_df.GDPPR_MAP_FLAG.sum()} out of {formatted_df.GDPPR_MAP_FLAG.count()} mapped SNOMED-CT ConceptIDs are available in the GDPPR dataset. Two codes 9T1Z.00 and 9T1A.00 have a ConceptID, however they are not clinically assured in the mapping file and have been discarded""")

171.0 out of 179 mapped SNOMED-CT ConceptIDs are available in the GDPPR dataset. Two codes 9T1Z.00 and 9T1A.00 have a ConceptID, however they are not clinically assured in the mapping file and have been discarded


In [11]:
formatted_df.count()

6_level_grouping         168
18_level_grouping        179
read_code                181
read_term                179
medcode                  179
READV2_MAP_FLAG          179
ConceptId                179
Cluster_Category         171
Cluster_ID               171
Cluster_Desc             171
ConceptId_Description    171
GDPPR_MAP_FLAG           179
dtype: int64

In [12]:
table_2 = pd.read_csv('output_data/table_2_snomedct_to_gdppr.csv')
table_2_cols_to_keep = ['level_of_certainty_of_migration_status', 
                        'type_of_code',
                        'read_code', 
                        'read_term',
                         'medcode',
                         'READV2_MAP_FLAG',
                         'ConceptId',
                         'Cluster_Category', 
                         'Cluster_ID',
                         'Cluster_Desc',
                         'ConceptId_Description',
                         'GDPPR_MAP_FLAG']

table_2 = table_2[table_2_cols_to_keep]

reseacher_codelist = get_project_code_list_2()
formatted_df = reseacher_codelist.merge(table_2, left_on ='read_code', right_on ='read_code', how='left', 
                         suffixes=('_original',''))

cols_to_drop = [i for i in formatted_df.columns if 'original' in i]
formatted_df.drop(columns = cols_to_drop, inplace=True)
formatted_df = formatted_df[table_2_cols_to_keep]

In [13]:
formatted_df.to_csv('output_data/formatted_table_2_snomedct_to_gdppr.csv', index=False)

In [14]:
formatted_df.nunique()

level_of_certainty_of_migration_status      3
type_of_code                                4
read_code                                 434
read_term                                 430
medcode                                   433
READV2_MAP_FLAG                             1
ConceptId                                 505
Cluster_Category                            1
Cluster_ID                                  2
Cluster_Desc                                2
ConceptId_Description                      18
GDPPR_MAP_FLAG                              2
dtype: int64

In [15]:
formatted_df.count()

level_of_certainty_of_migration_status    531
type_of_code                              531
read_code                                 532
read_term                                 531
medcode                                   531
READV2_MAP_FLAG                           531
ConceptId                                 531
Cluster_Category                           18
Cluster_ID                                 18
Cluster_Desc                               18
ConceptId_Description                      18
GDPPR_MAP_FLAG                            531
dtype: int64

In [16]:
formatted_df.sum()

/var/folders/h5/x2gkzm3j06z5k0dqwzgk7wvw0000gn/T/ipykernel_2506/1585014424.py:1: FutureWarning: The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  formatted_df.sum()


read_code          13e..0013gf.0013eG.0013eH.0013go.0013eY.0013gi...
medcode                                                   36492526.0
READV2_MAP_FLAG                                                531.0
ConceptId                                        57197434162630560.0
GDPPR_MAP_FLAG                                                  18.0
dtype: object

In [17]:
print(f"""{formatted_df.GDPPR_MAP_FLAG.sum()} out of {formatted_df.GDPPR_MAP_FLAG.count()} mapped SNOMED-CT ConceptIDs are available in the GDPPR dataset. The code 69D8.00 has a ConceptID, however is not clinically assured in the mapping file and has been discarded.""")

18.0 out of 531 mapped SNOMED-CT ConceptIDs are available in the GDPPR dataset. The code 69D8.00 has a ConceptID, however is not clinically assured in the mapping file and has been discarded.
